<a href="https://colab.research.google.com/github/naenumtou/statisticalModel/blob/main/faceTracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Set auto reload
%reload_ext autoreload
%autoreload 2

In [ ]:
# Change working directory
%cd /content/drive/My Drive/Colab Notebooks/faceTracking

/content/drive/My Drive/Colab Notebooks/faceTracking


In [ ]:
# Import libraries
import cv2
import numpy as np

# Config
%config InlineBackend.figure_format = 'retina' #Retina display

In [ ]:
# Utilise 'facenet' model for face detection
faceConfig = 'deploy.prototxt.txt'
faceModel = 'res10_300x300_ssd_iter_140000.caffemodel'

# Load 'facenet' model
faceNet = cv2.dnn.readNet(
    faceConfig,
    faceModel
)

# Face tracking
objectTracker = cv2.TrackerKCF_create() #Face tracking

In [ ]:
# Input video file
cap = cv2.VideoCapture('objectTracking.mp4')
w, h  = int(cap.get(3)), int(cap.get(4))

# Output
outVideo = cv2.VideoWriter(
    'trackingResult.avi',
    cv2.VideoWriter_fourcc(*'MJPG'), #With '.avi' format
    30,
    (w, h)
)

# Apply with video file
tracking = False

while True:
  ret, frame = cap.read()
  if ret == True:
    (fh, fw) = frame.shape[:2]
    if not tracking: #Tracking off
      blob = cv2.dnn.blobFromImage(
          frame,
          1.0,
          (300, 300),
          (104.0, 177.0, 123.0)
      )
      faceNet.setInput(blob)
      detections = faceNet.forward()
      for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence < 0.5:
          continue
        box = detections[0, 0, i, 3:7] * np.array([fw, fh, fw, fh])
        (startX, startY, endX, endY) = box.astype('int') #Return as the new area
        (startX, startY) = (max(0, startX), max(0, startY))
        (endX, endY) = (min(fw - 1, endX), min(fh - 1, endY))
        cv2.rectangle(
            frame,
            (startX, startY),
            (endX, endY),
            (0, 0, 255),
            2
        )
        if objectTracker.init(frame, (startX, startY, endX - startX, endY - startY)): #Minus to get location
          tracking = True #Found face --> on object tracker
    else: #Tracking on --> next iteration
      check, box = objectTracker.update(frame)
      if check: #Check track object
        # New location
        a = (int(box[0]), int(box[1])) #(x, y)
        b = (int(box[0] + box[2]), int(box[1] + box[3])) #(x + w, y + h)
        cv2.rectangle(
            frame,
            a,
            b,
            (0, 255, 0),
            2
        )
      else:
        tracking = False
        objectTracker = cv2.TrackerKCF_create()
    outVideo.write(frame)
  else:
    break